# Exercise 5 - Classification Algorithms (BBNN & SVM)

## a) Back Propogation Neural Network

### Program 1 - Implementing BPNN from scratch

#### AIM
To implement Back Propogation Neural Network from scratch in python.

#### ALGORITHM
##### Forward propogation
###### Single input
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\begin{align*}
\vect{o}_{-1} &= \vect{x}_i\\
\vect{o}_k &= \vect{\phi}_k\left(\vect{W}_k[1 \ \vect{o}_{k-1}]\right) &\text{for }k = 0,..,n-1
\end{align*}
$$
###### Batch input
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\begin{align*}
\vect{O}_{-1} &= \vect{X}\\
\vect{O}_k &= \vect{\phi}_k\left([\vect{1} \ \vect{O}_{k-1}] \vect{W}_k^T \right) &\text{for }k = 0,..,n-1
\end{align*}
$$

where  
$\vect{x}_k$ - the input vector  
$\vect{o}_k$ - the output vector of the $k$th layer  
$\vect{W}_k$ - the weight matrix of the $k$th layer
$\vect{X}_k$ - the batch input matfix  
$\vect{O}_k$ - the batch output matrix of the $k$th layer  

##### Backward Propogation
###### Error
$$
E = \frac{1}{2}||\vect{o}_{n-1}-\vect{t}||_2
$$
###### Single input
$$
\newcommand{\diff}[2]{\frac{\mathrm{d}{#1}}{\mathrm{d}{#2}}}
\vect{\delta}_{k} = \diff{E}{\vect{o}_{k}}\circ \vect{\phi}'(\text{net}_{k} )\\
$$

$$
\begin{gather*}
\diff{E}{\vect{o}_{n-1}} = (\vect{o}_{n-1}-\vect{t})  
& \diff{E}{\vect{o}_{k}} = \vect{W}^T_{k+1}\vect{o}_{k+1},\ \text{for } k = n-2,...,-1 \\
\end{gather*}
$$

$$
\vect{\phi}'(\text{net}_{k}) = \vect{o}_k \circ (\vect{1}-\vect{o}_k),\ \text{for } k = n-1,...,0 \\
$$
Therefore,
$$
\vect{\delta}_{n-1} = (\vect{o}_{n-1}-\vect{t}) \circ \vect{o}_{n-1} \circ (\vect{1}-\vect{o}_{n-1})\\
\vect{\delta}_{k} = \vect{W}^T_{k+1}\vect{o}_{k+1} \circ \vect{o}_k \circ (\vect{1}-\vect{o}_k),\ \text{for } k = n-1,...,0 \\\\
$$

###### Weight Update
$$
\vect{W}_k := \vect{W}_k - \alpha \diff{E}{\vect{W}_k}\\
\diff{E}{\vect{W}_k} = \vect{\delta}_{k}[1 \ \vect{o}_{k-1}]^T,\ \text{for } k = n-1,...,0 \\
$$

#### Part 1 - Defining class for BPNN

In [1]:
import numpy as np
import pandas as pd

In [2]:
class BPNN:
    def __init__(self,layer_sizes):
        self.layer_sizes = layer_sizes
        self.n = len(layer_sizes)-1
        self.outputs = [None]*(self.n+1)
        self.delta = [None]*self.n
        
    def error(self,predicted, actual):
        error = actual-predicted
        return error @ error.T
    
    def activation(self,x):
        return 1/(1+np.exp(-x))
    
    def d_activation(self,x):
         return x * (1 - x)
    
    def pad_ones(X):
        pad_width = [(0,0)]*(X.ndim-1) +[(1,0)]
        return np.pad(X,pad_width=pad_width,constant_values=1)
    
    def inputs(self,i):
        return BPNN.pad_ones(self.outputs[i-1])
    
    def set_random_weights(self,seed=1):
        np.random.seed(seed)
        self.weights = [
            (np.random.random((o,i+1))-1)*2
            for i,o in zip(self.layer_sizes,self.layer_sizes[1:])
        ]

    def forward_propagate(self, inputs):
        assert (self.weights is not None),"weights not given"
        self.outputs[-1] = inputs
        for i in range(self.n):
            self.outputs[i] = self.activation(
                self.inputs(i) @ self.weights[i].T
            )
        return self.outputs[self.n-1]

    def backward_propagate_error(self, expected):
        for i in range(self.n-1, -1, -1):
            if i == self.n-1:
                errors = self.outputs[i] - expected
            else:
                errors = self.delta[i+1] @ self.weights[i+1][:,1:] 
            self.delta[i] = errors * self.d_activation(self.outputs[i])

    def update_weights(self, l_rate):
        for i in range(self.n):
            self.weights[i] -= l_rate * self.delta[i].T @ self.inputs(i)

    def fit(self, X_train,y_train, l_rate, n_epoch,batch_size=10,verbose=True):
        classes = np.unique(y_train)
        expected = (y_train.reshape(-1,1) == classes).astype(np.uint8)
        m = X_train.shape[0]
        for epoch in range(n_epoch):
            sum_error = 0
            b_idx = np.arange(0,m+batch_size,batch_size)
            for f,t in zip(b_idx,b_idx[1:]):
                outputs = self.forward_propagate(X_train[f:t])
                sum_error += np.sum(self.error(outputs, expected[f:t]))
                self.backward_propagate_error(expected[f:t])
                self.update_weights(l_rate)
            if verbose:
                print(f'>epoch={epoch}, lrate={l_rate:.3},'
                      f' error={sum_error/m:.5f}')

    def predict(self, inputs):
        outputs = self.forward_propagate(inputs).argmax(axis=-1)
        return outputs

    def accuracy(self,X_test,y_test):
        return (self.predict(X_test)==y_test).mean()

#### Part 2 - Loading and processing dataset

In [3]:
titanic_df = pd.read_csv("datasets/titanic_processed.csv")
X = titanic_df.drop('Survived',axis = 1).values
y = titanic_df['Survived'].values
split_ratio = 0.8
s = int(split_ratio*X.shape[0])
X_train, X_test = X[:s],X[s:]
y_train, y_test = y[:s],y[s:]

#### Part 3 - Implementing BPNN

In [4]:
b = BPNN([8,5,2])
b.set_random_weights(10)
print("\nInitial Weights:\n")
print(*b.weights,sep="\n")
print("\nTraining:\n")
b.fit(X_train,y_train,l_rate=.5, n_epoch=20, batch_size=50)
print("\nTrained Weights:\n")
print(*b.weights,sep="\n")
print("\nEvaluation:")
print("Acuracy of the classifier: ",b.accuracy(X_test,y_test))


Initial Weights:

[[-0.45735871 -1.9584961  -0.73270353 -0.50239223 -1.00298598 -1.55040671
  -1.60387427 -0.47893858 -1.66177833]
 [-1.82332037 -0.62928036 -0.09321331 -1.99210347 -0.97561547 -0.37475808
  -0.77494787 -0.55648937 -1.41624786]
 [-0.16445175 -0.57084843 -0.91491126 -1.7156599  -1.25331848 -0.65173277
  -1.11633365 -1.13197201 -0.76446604]
 [-0.97372351 -0.69920564 -0.79792209 -0.38955361 -0.9567057  -0.18270224
  -1.36152782 -1.8190813  -1.39859989]
 [-1.77203128 -0.34263735 -1.90620736 -0.7474257  -0.90482769 -0.36142601
  -1.60210492 -0.2862994  -1.29669472]]
[[-0.49070462 -1.40807659 -0.23212704 -1.34897672 -1.6699682  -1.21494151]
 [-1.81307925 -0.35778868 -1.69769596 -1.2317711  -0.11147858 -0.02474905]]

Training:

>epoch=0, lrate=0.5, error=1.69117
>epoch=1, lrate=0.5, error=1.37732
>epoch=2, lrate=0.5, error=3.52303
>epoch=3, lrate=0.5, error=3.94931
>epoch=4, lrate=0.5, error=2.96108
>epoch=5, lrate=0.5, error=2.18476
>epoch=6, lrate=0.5, error=2.45861
>epoch=

## b) Support Vector Machine

### Program 1 - Implementing SVM from scratch using CVXOPT

#### AIM
To implement SVM from scratch using CVXOPT.

#### Formula
##### SVM statement in dual form
$$
\newcommand{\vect}[1]{\boldsymbol{\mathbf{#1}}}
\begin{aligned}
    & \min_{\vect{\alpha}}  \frac{1}{2}  \vect{\alpha}^T  \left( \vect{y}\vect{y}^T \circ (\Phi(\vect{X})\Phi(\vect{X})^T \right)  \vect{\alpha} - \vect{1}^T \vect{\alpha}\\
s.t.&  - \alpha_i \leq 0 \\
    & \alpha_i \leq C\\
    & y^T \vect{\alpha} = 0  
\end{aligned}
$$

##### Standard form of a quadratic program
$$
\begin{aligned}
    & \min_{\vect{x}}  \frac{1}{2}  \vect{x}^T  \vect{P}  \vect{x} - \vect{q}^T\vect{x}\\
s.t.&  \vect{G}\vect{x} \leq \vect{h} \\
    & A \vect{x} = \vect{b} 
\end{aligned}
$$

##### Formulization
$$
\begin{align*}
\vect{P}&= \vect{y}\vect{y}^T \circ \Phi(\vect{X})\Phi(\vect{X})^T
& \vect{q}&= - \vect{1}_n\\
\vect{G}&= \begin{bmatrix}
    \vect{-I}_n\\
    \vect{I}_n
\end{bmatrix} 
& \vect{h}&=\begin{bmatrix}
    C \cdot \vect{1}_n^T\\
    \vect{0}_n^T
\end{bmatrix}\\
\vect{A} &= [\vect{y}] 
&\vect{b} &= [0]\\
\end{align*}
$$

##### Decision Rule
$$
\vect{\hat y} = \mathrm{sign}\left( (\vect{\alpha}^T \circ \vect{y}^T) \Phi(\vect{X})\Phi(\vect{X})^T  + b \right) 
$$

#### Part 1 - Defining class for SVM

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from cvxopt import solvers
from cvxopt import matrix

from scipy.spatial.distance import cdist

In [6]:
class SVM:
    def __init__(self,C,kernel):
        self.C = C
        self.kernel = kernel

    def fit(self,X_train,y_train):
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(X_train)
        self.y = y_train.reshape(-1,1)
        
        n=self.X.shape[0]
        I_n = np.eye(n)
        P=(self.y@self.y.T)*self.kernel(self.X,self.X)
        q=np.full(n,-1)
        G=np.vstack((I_n,-1*I_n))
        h=np.hstack((np.full(n,self.C),np.zeros(n)))
        A=y_train.reshape(1,-1)
        b=np.zeros(1)

        P,q,G,h,A,b = map(lambda x : matrix(x,tc="d"),(P,q,G,h,A,b))

        solution = solvers.qp(P, q, G, h, A, b)
        self.a = np.asarray(solution['x']).squeeze()
        
        support_indices = np.logical_and(self.a>=1e-10, self.a<self.C)
        X_S = self.X[support_indices]
        self.b = np.mean(self.y - self.a*self.y.T @ self.kernel(self.X, X_S))

    def predict(self,X_test):
        X_test=self.scaler.transform(X_test)
        return np.sign(self.a*self.y.T @ self.kernel(self.X, X_test) + self.b)


#### Part 2 - Defining Radial Basis Function(RBF) Kernel

In [7]:
def rbf_kernel(X1,X2,sigma):
    return np.exp(-cdist(X1, X2, 'sqeuclidean') / (2*sigma**2))

#### Part 3 - Loading and Processing Dataset

In [8]:
titanic_df = pd.read_csv('datasets/titanic_processed.csv')
X = titanic_df.drop('Survived',axis = 1).values
y = titanic_df['Survived'].values
y[y==0] = -1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Part 4 - Implementing SVM

In [9]:
from functools import partial 
C = .35
sigma = .5
kernel = partial(rbf_kernel,sigma=sigma)

svm_classifier = SVM(C,kernel)
svm_classifier.fit(X_train,y_train)
y_pred = svm_classifier.predict(X_test)
print(f'Accuracy of the classifer: {(y_test == y_pred).mean()*100:.2f}%')


     pcost       dcost       gap    pres   dres
 0: -2.0672e+02 -6.0489e+02  6e+03  8e+00  1e-15
 1: -1.0821e+02 -5.1411e+02  6e+02  3e-01  1e-15
 2: -1.0989e+02 -1.6099e+02  5e+01  9e-03  2e-15
 3: -1.2250e+02 -1.3135e+02  9e+00  1e-03  1e-15
 4: -1.2441e+02 -1.2830e+02  4e+00  4e-04  1e-15
 5: -1.2525e+02 -1.2680e+02  2e+00  1e-04  1e-15
 6: -1.2558e+02 -1.2625e+02  7e-01  7e-06  1e-15
 7: -1.2572e+02 -1.2596e+02  2e-01  9e-07  1e-15
 8: -1.2577e+02 -1.2588e+02  1e-01  4e-07  1e-15
 9: -1.2578e+02 -1.2586e+02  8e-02  2e-07  9e-16
10: -1.2580e+02 -1.2582e+02  2e-02  2e-08  1e-15
11: -1.2581e+02 -1.2581e+02  6e-03  6e-09  1e-15
12: -1.2581e+02 -1.2581e+02  3e-03  1e-09  1e-15
13: -1.2581e+02 -1.2581e+02  9e-04  3e-10  9e-16
14: -1.2581e+02 -1.2581e+02  3e-04  9e-11  1e-15
15: -1.2581e+02 -1.2581e+02  2e-05  4e-15  1e-15
Optimal solution found.
Accuracy of the classifer: 73.03%
